<img align="left" src = https://noirlab.edu/public/media/archives/logos/svg/logo250.svg width=250 style="background-color:white; padding-right:10px; margin-top:-70px; margin-bottom:-70px;" alt="Rubin Observatory logo, a graphical representation of turning stars into data.">

**Description:** Generate property map plots for _Rubin Observatory DP1_ paper <br>
**Contact authors:** Erfan Nourbakhsh <br>
**Last verified to run:** 2025-05-06 <br>
**LSST Science Piplines version:** w_2025_18 <br>
**Container Size:** Medium (or larger) <br>
**Location:** This notebook points to files on the S3DF cluster at the USDF. Update paths accordingly if you are running elsewhere.

### Import required libraries and modules

In [1]:
import os
from lsst.daf.butler import Butler, DataCoordinate
from lsst.analysis.tools.atools import PerTractPropertyMapTool, SurveyWidePropertyMapTool
from lsst.analysis.tools.tasks.propertyMapAnalysis import PerTractPropertyMapAnalysisConfig, PerTractPropertyMapAnalysisTask

from pathlib import Path

### Set up the paths, data IDs, and map names

In [2]:
# Create a directory to store the output plots.
outputDir = str(Path('../figures/dp1_property_maps').resolve())
outputFormat = "pdf"
os.makedirs(outputDir, exist_ok=True)

In [3]:
# Define the repository and collection to be used.
repo = "/repo/dp1"
collection = "LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260"
skymapName = "lsst_cells_v1"

# Make a list for dictionaries of data ID values.
dataIds = [
    {"band": "g", "skymap": skymapName, "tract": 5063},
    {"band": "z", "skymap": skymapName, "tract": 5063},
    {"band": "r", "skymap": skymapName, "tract": 10463},
]

# Make a list of survey property maps to be plotted in the same order as the
# data IDs above. These are the ones we used as examples in the DP1 paper.
mapNames = [
    "deepCoadd_sky_noise_map_weighted_mean",
    "deepCoadd_psf_maglim_map_weighted_mean",
    "deepCoadd_exposure_time_map_sum",
]

# Set this to False if you only want the `mapNames` above to be plotted for the
# corresponding data IDs.
plotAllConfiguredMaps = False

### Load data for processing and configure the tool

In [4]:
# Create a butler object to access the data.
butler = Butler(repo, collections=collection)

In [5]:
# Configure the plotter.
config = PerTractPropertyMapAnalysisConfig()
config.projectionKwargs = {"celestial": True, "gridlines": True, "min_lon_ticklabel_delta": 0.12}
config.colorbarKwargs = {"cmap": "viridis"}
config.publicationStyle = True

In [6]:
# Configure the maps to potentially be plotted. If you want all these maps to
# be plotted, regardless of `mapNames`, set `plotAllConfiguredMaps` to True.
# The entries in the 'atools' namespace must exactly match the dataset types.
config.atools.deepCoadd_dcr_ddec_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_dcr_dra_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_dcr_e1_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_dcr_e2_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_epoch_map_min = PerTractPropertyMapTool()
config.atools.deepCoadd_epoch_map_max = PerTractPropertyMapTool()
config.atools.deepCoadd_epoch_map_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_exposure_time_map_sum = PerTractPropertyMapTool()
config.atools.deepCoadd_psf_e1_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_psf_e2_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_psf_maglim_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_psf_size_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_sky_background_map_weighted_mean = PerTractPropertyMapTool()
config.atools.deepCoadd_sky_noise_map_weighted_mean = PerTractPropertyMapTool()

### Loop over the data IDs and save the plot for each one

In [7]:
# config.atools.fieldNames
for dId, mapName in zip(dataIds, mapNames):
    # Standardize the data ID.
    dataId = DataCoordinate.standardize(dId, universe=butler.dimensions)
    print(dataId)

    for name, atool in zip(config.atools.fieldNames, config.atools):
        if not plotAllConfiguredMaps and name != mapName:
            continue
    
        # Finalize the tool to ensure it is ready for use.
        atool.finalize()
    
        # Match task input format: {datasetTypeName: DeferedDataSetHandle}.
        inputs = {}
        inputs[name] = butler.getDeferred(name, dataId=dataId)
    
        # Get the tract information from the skymap.
        skymap = butler.get("skyMap", dataId=dataId)
        tract = dataId["tract"]
        tractInfo = skymap[tract]
    
        # Even in publication style, where we limit plot details, we still need
        # to provide basic plot info for minimal annotations.
        plotInfo = PerTractPropertyMapAnalysisTask().parsePlotInfo(inputs, dataId, [name])
    
        # Use the selected tool to process the data.
        results = atool(data=inputs, tractInfo=tractInfo, plotConfig=config, plotInfo=plotInfo)
    
        # Pull the figure from the results dictionary and save it.
        fig = results[f"{name}_{atool.produce.plot.getPlotType()}"]
        fig.savefig(figName := f"{outputDir}/{name}_tract{tract}_{dataId['band']}band.{outputFormat}")
        print(f"Saved plot to {figName}")

{band: 'g', skymap: 'lsst_cells_v1', tract: 5063}


Saved plot to /sdf/data/rubin/user/lguy/lsst/repos/rtn-095/figures/dp1_property_maps/deepCoadd_sky_noise_map_weighted_mean_tract5063_gband.pdf
{band: 'z', skymap: 'lsst_cells_v1', tract: 5063}


Saved plot to /sdf/data/rubin/user/lguy/lsst/repos/rtn-095/figures/dp1_property_maps/deepCoadd_psf_maglim_map_weighted_mean_tract5063_zband.pdf
{band: 'r', skymap: 'lsst_cells_v1', tract: 10463}


Saved plot to /sdf/data/rubin/user/lguy/lsst/repos/rtn-095/figures/dp1_property_maps/deepCoadd_exposure_time_map_sum_tract10463_rband.pdf
